<a href="https://colab.research.google.com/github/Tonge-Shim/1/blob/master/sgd%26forest_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import DataFrame
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(max_iter=5, random_state=42)

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
df= pd.read_csv('samp_cst_feat.csv')
dtype = pd.read_excel('variable_dtype.xlsx')
dtype_categ = dtype[(dtype['dType']=='categorical')]
dtype_numrc = dtype[(dtype['dType']=='numerical')]
numrc_var = dtype_numrc['Variable_Name']
categ_var = dtype_categ['Variable_Name']
numrc_var = numrc_var.values.tolist()
categ_var = categ_var.values.tolist()#0,1
x=df.drop(categ_var, axis=1).drop(['cst_id_di'],axis=1)#10124개
y=pd.read_csv('samp_train.csv').drop(['cst_id_di'],axis=1)#10124
y_nozero= y[(y['MRC_ID_DI']!=0)]#not 0!!!1924
#y_nozeroindex=y_nozero.loc[1:10119]
y_nozeroindexlist=y_nozero.index.tolist()
x_nozero=x.loc[y_nozeroindexlist]

In [3]:
print(x_nozero.shape)
print(y_nozero.shape)
print(x.shape)
print(y.shape)
y_nozeroarray=y_nozero.to_numpy()#1924
y_array=y.to_numpy()#10124
x_nozeroarray=x_nozero.to_numpy()#1924
x_array=x.to_numpy()#10124

(1924, 198)
(1924, 1)
(10124, 198)
(10124, 1)


#0을 제외한 1924개

In [5]:
x_train, x_test, y_train, y_test=x_nozeroarray[:1400], x_nozeroarray[1400:], y_nozeroarray[:1400], y_nozeroarray[1400:]
shuffle_index=np.random.permutation(1400)
x_train, y_train=x_train[shuffle_index], y_train[shuffle_index]

#0기준 이진분류

In [36]:
X_train, X_test, Y_train, Y_test=x_array[:9000], x_array[9000:], y_array[:9000], y_array[9000:]
shuffle_index=np.random.permutation(9000)
X_train, Y_train=X_train[shuffle_index], Y_train[shuffle_index]

Y_train_0=(Y_train==0)#0은 true, 0이 아니면 false
Y_test_0=(Y_test==0)



#0제외한 1924데이터 분류

In [61]:
y_1=y[(y['MRC_ID_DI']==1)]
y_2=y[(y['MRC_ID_DI']==2)]
y_3=y[(y['MRC_ID_DI']==3)]
y_4=y[(y['MRC_ID_DI']==4)]
y_5=y[(y['MRC_ID_DI']==5)]
y_6=y[(y['MRC_ID_DI']==6)]
y_7=y[(y['MRC_ID_DI']==7)]
y_8=y[(y['MRC_ID_DI']==8)]
y_9=y[(y['MRC_ID_DI']==9)]
y_10=y[(y['MRC_ID_DI']==10)]

x_1=x.loc[y_1.index.tolist()]
x_2=x.loc[y_2.index.tolist()]
x_3=x.loc[y_3.index.tolist()]
x_4=x.loc[y_4.index.tolist()]
x_5=x.loc[y_5.index.tolist()]
x_6=x.loc[y_6.index.tolist()]
x_7=x.loc[y_7.index.tolist()]
x_8=x.loc[y_8.index.tolist()]
x_9=x.loc[y_9.index.tolist()]
x_10=x.loc[y_10.index.tolist()]

In [66]:
y_nozero.value_counts()

MRC_ID_DI
7            661
6            554
8            337
5            143
1             85
9             59
3             32
10            28
2             14
4             11
dtype: int64

#5,6,7,8 분류 55%정확도

In [83]:
x_5678=x.loc[y_5.index.tolist()+y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_5678=y.loc[x_5678.index.tolist()]
print(x_5678.shape)
print(y_5678.shape)
x_5678=x_5678.to_numpy()
y_5678=y_5678.to_numpy()

(1695, 198)
(1695, 1)


In [84]:
x_5678train, x_5678test, y_5678train, y_5678test=x_5678[:1250], x_5678[1250:], y_5678[:1250], y_5678[1250:]
shuffle_index=np.random.permutation(1250)
x_5678train, y_5678train=x_5678train[shuffle_index], y_5678train[shuffle_index]
some_digit=x_5678[3]#y_5678[3]=5

In [102]:
sgd_clf.fit(x_5678train, y_5678train)
sgd_clf.predict([some_digit])

array([7])

In [103]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.57074341, 0.58992806, 0.43990385])

In [104]:
from sklearn.ensemble import RandomForestClassifier#제일 나음
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_5678train, y_5678train)
forest_clf.predict([some_digit])

array([5])

In [105]:
cross_val_score(forest_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.61390887, 0.57074341, 0.63221154])

In [106]:
from sklearn.multiclass import OneVsOneClassifier
ovo_clf=OneVsOneClassifier(SGDClassifier(max_iter=5, random_state=42))
ovo_clf.fit(x_5678train, y_5678train)
ovo_clf.predict([some_digit])

array([6])

In [107]:
cross_val_score(ovo_clf, x_5678train, y_5678train, cv=3, scoring="accuracy")

array([0.55155875, 0.47242206, 0.57692308])

이건 아니지이이이잉

#678 분류 67%정확도

In [87]:
x_678=x.loc[y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()]
y_678=y.loc[x_678.index.tolist()]
print(x_678.shape)
print(y_678.shape)
x_678=x_678.to_numpy()
y_678=y_678.to_numpy()

(1552, 198)
(1552, 1)


In [111]:
x_678train, x_678test, y_678train, y_678test=x_678[:1230], x_678[1230:], y_678[:1230], y_678[1230:]
shuffle_index=np.random.permutation(1230)
x_678train, y_678train=x_678train[shuffle_index], y_678train[shuffle_index]
some_digit1=x_678[3]#y_678[3]=6

In [112]:
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_678train, y_678train)
forest_clf.predict([some_digit1])

array([6])

In [114]:
cross_val_score(forest_clf, x_678train, y_678train, cv=3, scoring="accuracy")

array([0.66341463, 0.67560976, 0.67073171])

In [121]:
y.loc[y_6.index.tolist()+y_7.index.tolist()+y_8.index.tolist()].value_counts()

MRC_ID_DI
7            661
6            554
8            337
dtype: int64


661/(661+554+337)*100=43

554/(661+554+337)*100=35.7

337/(661+554+337)*100=21.7




#6,7분류 약 70%정확도


In [125]:
x_67=x.loc[y_6.index.tolist()+y_7.index.tolist()]
y_67=y.loc[x_67.index.tolist()]
print(x_67.shape)
print(y_67.shape)
x_67=x_67.to_numpy()
y_67=y_67.to_numpy()

(1215, 198)
(1215, 1)


In [127]:
x_67train, x_67test, y_67train, y_67test=x_67[:950], x_67[950:], y_67[:950], y_67[950:]
shuffle_index=np.random.permutation(950)
x_67train, y_67train=x_67train[shuffle_index], y_67train[shuffle_index]
some_digit1=x_67[3]#y_67[3]=6

In [128]:
#randomforest
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_67train, y_67train)
forest_clf.predict([some_digit1])

array([6])

In [129]:
cross_val_score(forest_clf, x_67train, y_67train, cv=3, scoring="accuracy")


array([0.70977918, 0.68138801, 0.69936709])

In [131]:
#sgd_clf
sgd_clf.fit(x_67train, y_67train)
sgd_clf.predict([some_digit1])

array([6])

In [132]:
cross_val_score(sgd_clf, x_67train, y_67train, cv=3, scoring="accuracy")

array([0.53943218, 0.60252366, 0.61708861])

In [135]:
y_nozero.value_counts()

MRC_ID_DI
7            661
6            554
8            337
5            143
1             85
9             59
3             32
10            28
2             14
4             11
dtype: int64

#2,3,4,9,10

In [137]:
x_210=x.loc[y_2.index.tolist()+y_3.index.tolist()+y_4.index.tolist()+y_9.index.tolist()+y_10.index.tolist()]
y_210=y.loc[x_210.index.tolist()]
print(x_210.shape)
print(y_210.shape)
x_210=x_210.to_numpy()
y_210=y_210.to_numpy()

x_210train, x_210test, y_210train, y_210test=x_210[:110], x_210[110:], y_210[:110], y_210[110:]
shuffle_index=np.random.permutation(110)
x_210train, y_210train=x_210train[shuffle_index], y_210train[shuffle_index]
some_digit1=x_210[3]#y_210[3]=2

(144, 198)
(144, 1)


In [141]:
#randomforest
forest_clf=RandomForestClassifier(random_state=42)
forest_clf.fit(x_210train, y_210train)
forest_clf.predict([some_digit1])

array([2])

In [142]:
cross_val_score(forest_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.56756757, 0.56756757, 0.52777778])

In [143]:
#sgd_clf
sgd_clf.fit(x_210train, y_210train)
sgd_clf.predict([some_digit1])
cross_val_score(sgd_clf, x_210train, y_210train, cv=3, scoring="accuracy")


array([0.62162162, 0.43243243, 0.5       ])